In [2]:
import vertica_python
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandasql as ps
from pandasql import sqldf
import time

In [3]:
def run_vrtc_query(query=None):
    
    """Queries data from Vertica and returns pandas dataframe"""

    # The user and password here are my own. 
    # The password will most probably be needed to be updated here, 
    # everytime I update it on my computer account. 
    vertica_conn = {
        'host': 'casual-prod-vertica-eon-lb.va2',
        'port': 5433,
        'user': 'alexanderloe',
        'password': 'Worlds2022!',
        'database': 'calamari',}
    

    if not query:
        return
    try:
        connection = vertica_python.connect(**vertica_conn)
        with connection.cursor() as cursor:
            cursor.execute(query)
            names = [x[0] for x in cursor.description]
            return pd.DataFrame(cursor.fetchall(), columns=names)
        
    except Exception as err:
        print('Error: ', err)
        exit(1)

In [4]:
def load_data_from_vertica(sql):
    df = run_vrtc_query(sql)
    print("done vrtc_query")
    return df

In [5]:
# Set Pandas to display more columns as default
pd.options.display.max_columns=60

In [6]:
date_train_end = 20220915
query_data_for_clustering_check_features = f"""select * from analytics.agg_bb_daily_user_measures_snapshot as a
    where a.dim_date_gid = '{date_train_end}' limit 10;"""

In [7]:
t0= time.time()

data_for_clustering_check_features = load_data_from_vertica(query_data_for_clustering_check_features)

t1 = time.time()
print("\nTime elapsed: ", t1-t0)

print()
print(data_for_clustering_check_features.shape)
data_for_clustering_check_features.head(10)

done vrtc_query

Time elapsed:  2.2648839950561523

(10, 155)


,dim_user_gid,dim_date_gid,date,is_in_DAU,top_platform,top_device,top_login_method,top_login_country,level,trstier,level_bracket,is_elite,is_reactviated_user,is_reactviated_payer,is_vip,is_notifications_enabled,number_of_platforms,number_of_devices,number_of_sessions,time_on_game_seconds,time_bingo_rooms_seconds,time_bingo_rooms_with_tournament_seconds,amount_of_push_notifications_received,last_login_date,last_gameplay_date,max_app_version,max_mobile_appversion,Is_fb_connected,Is_one_connected,Is_apple_connected,...,gross_sink_Specialty_Room_rounds,total_rounds_mode_6,Last_60D_Gross_Sink,Last_60D_Login_days,Last_60D_gameplay_days,Last_30D_rounds,Last_60D_rounds,Last_90D_rounds,is_free_trial_period,Life_Time_Login_Days,credits_finish_4,time_left_z,gems_balance_EOD,gems_spent,last_30D_gems_spent,last_90D_gems_spent,Lifetime_gems_spent,gems_transaction,last_30D_Gems_transactions,last_90D_gems_transactions,lifetime_gems_transactions,gems_purchased,last_30D_gems_purchased,last_90D_gems_purchased,gems_purchased_tx,last_30D_gems_purchased_tx,last_90D_gems_purchased_tx,lifetime_gems_purchased_tx,gems_inflow,lifetime_gems_purchased
0,600,20220915,2022-09-15,1,PRAS,PC,Facebook,UNITED STATES,546,6,lv_200plus,1,0,0,1,-1,1,1,2,2167,690,690,0,2022-09-14 09:30:23.683,2022-09-14 09:36:13,5010,4722,1,0,0,...,0,0,246360.0000,60,35,178,277,473,0,1667,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,800,20220915,2022-09-15,1,Facebook,PC,Facebook,CANADA,266,2,lv_200plus,0,0,0,0,-1,1,1,3,1955,751,751,0,2022-09-14 22:22:32.526,2022-09-14 22:31:51,5010,None,1,0,0,...,0,0,37350.0000,60,53,373,713,1066,0,2459,18,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1223,20220915,2022-09-15,1,PRAS,PC,Facebook,UNITED STATES,482,5,lv_200plus,1,0,0,1,-1,1,1,1,3392,1887,1887,0,2022-09-14 20:58:27.560,2022-09-14 21:25:56,5000,4890,1,0,0,...,0,0,194636.0000,60,55,480,1088,1741,0,2532,18,15,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,1465,20220915,2022-09-15,1,Google,Android Phone,Facebook,ITALY,244,3,lv_200plus,0,0,0,0,1,1,1,1,1384,0,0,0,2022-09-14 23:00:19.816,2022-09-14 23:15:09,5000,5000,1,0,0,...,0,0,27275.0000,12,10,39,119,119,0,1522,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,1510,20220915,2022-09-15,1,Facebook,PC,Facebook,UNITED STATES,242,4,lv_200plus,1,0,0,0,-1,1,1,2,6704,3794,3794,0,2022-09-14 20:52:28.528,2022-09-14 21:05:44,5010,5000,1,0,0,...,0,0,238175.0000,60,60,1347,3179,4361,0,1230,30,4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,1646,20220915,2022-09-15,1,Google,Android Phone,Facebook,AUSTRALIA,314,4,lv_200plus,0,0,0,0,1,1,1,1,1779,873,873,0,2022-09-14 16:39:24.512,2022-09-14 06:22:54,5000,5000,1,0,0,...,0,0,131151.0000,54,36,153,333,461,0,1391,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,1735,20220915,2022-09-15,1,Apple,iPad,Facebook Limited,UNITED KINGDOM,186,2,lv_170_199,0,0,0,0,1,1,1,2,5699,995,1487,0,2022-09-14 19:34:25.711,2022-09-14 09:08:17,5002,5002,0,0,0,...,0,0,28080.0000,54,49,250,407,515,0,352,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,2191,20220915,2022-09-15,1,Google,Android Phone,Facebook,UNITED KINGDOM,291,2,lv_200plus,0,0,0,0,1,1,1,4,2245,677,677,0,2022-09-14 21:29:51.299,2022-09-14 20:04:14,5000,5000,1,0,0,...,0,0,203340.0000,60,60,878,2124,2380,0,2336,20,13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,2447,20220915,2022-09-15,1,PRAS,PC,Facebook,AUSTRALIA,685,5,lv_200plus,1,0,0,1,-1,1,1,2,6620,2433,2433,0,2022-09-14 23:57:15.361,2022-09-13 23:24:49,5010,4960,1,0,0,...,0,4,2391415.0000,60,54,1183,1934,2941,0,1604,30,12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,2472,20220915,2022-09-15,1,Google,An

In [27]:
columns_names = data_for_clustering_check_features.columns.values.tolist()
check_words = ['login', 'first', 'life', 'lifetime', 'date']
for item in columns_names:
    if any(word in item.split("_") for word in check_words): 
        print(item)

dim_date_gid
date
top_login_method
top_login_country
last_login_date
last_gameplay_date
distinct_login_channels
first_login_origin_type
first_login_origin_id
lifetime_gems_transactions
lifetime_gems_purchased_tx
lifetime_gems_purchased


In [6]:
date_train_end_format_full = "2022-09-15"
date_train_end = 20220915

date_target = 20220916

In [7]:
# The query for the data for the clustering
query_data_for_clustering = f"""select dim_user_gid,
    top_platform,
    level,
    trstier,
    level_bracket,
    is_elite,
    is_vip,
    datediff(day, last_login_date, '{date_train_end_format_full}') as llrecency,
    Lifetime_Revenue_Net,
    Last_30D_Revenue_Net,
    Last_90D_Revenue_Net,
    Last_30D_Transactions,
    Last_30D_Gross_Sink,
    Last_90D_Gross_Sink,
    Last_30D_Login_days,
    Last_90D_Login_days,
    credits_balance_EOD,
    coins_balance_EOD,
    number_of_bingo_rounds,
    Life_Time_Login_Days,
    total_rounds_mode_1,
    total_rounds_mode_2,
    total_rounds_mode_3,
    total_rounds_mode_4,
    total_rounds_mode_5,
    total_rounds_mode_6
    from analytics.agg_bb_daily_user_measures_snapshot as a
    where a.dim_date_gid = '{date_train_end}' and a.last_login_date <= '{date_train_end_format_full}'
    limit 5000000;"""


In [5]:
# An example for a target with numerical values 
query_target_revenue = f"""select dim_user_gid, revenue_converted_net
from blitz_all.core_transaction_purchase
where dim_date_gid = '{date_target}';"""

In [6]:
# An example for a target with discrete values 
query_target_level_ups = f"""select dim_user_gid, count(*) as level_ups
from blitz_all.blitz_level_up
where dim_date_gid = '{date_target}'
group by 1;"""

In [7]:
# An example for a target with binary values 
query_target_mode_number = f"""select dim_user_gid, round(avg(modenumber)) as typical_modenumber
from blitz.blitz_bingo_round_start
where dim_date_gid = '{date_target}' and modenumber >= 1
group by 1;"""

In [8]:
t0= time.time()

data_for_clustering = load_data_from_vertica(query_data_for_clustering)

t1 = time.time()
print("\nTime elapsed: ", t1-t0)

print()
print(data_for_clustering.shape)
data_for_clustering.head(10)

done vrtc_query

Time elapsed:  1056.6003980636597

(5000000, 26)


,dim_user_gid,top_platform,level,trstier,level_bracket,is_elite,is_vip,llrecency,Lifetime_Revenue_Net,Last_30D_Revenue_Net,...,credits_balance_EOD,coins_balance_EOD,number_of_bingo_rounds,Life_Time_Login_Days,total_rounds_mode_1,total_rounds_mode_2,total_rounds_mode_3,total_rounds_mode_4,total_rounds_mode_5,total_rounds_mode_6
0,953,Web,272.0,2.0,lv_200plus,0,0,1,0.0000,0.0000,...,36018.0,16136268.0,7,2335,0,0,3,3,1,0
1,1054,Google,226.0,4.0,lv_200plus,1,0,1,83.7783,12.8209,...,8009.0,1520319.0,32,903,0,32,0,0,0,0
2,1103,Google,168.0,2.0,lv_140_169,0,0,1,0.0000,0.0000,...,31133.0,7017099.0,0,1226,0,0,0,0,0,0
3,1147,Apple,485.0,4.0,lv_200plus,0,0,1,1196.0878,0.0000,...,26329.0,3637203.0,0,2492,0,0,0,0,0,0
4,1148,Facebook,227.0,2.0,lv_200plus,0,0,1,0.0000,0.0000,...,11769.0,45457627.0,15,2228,0,10,0,0,0,0
5,1962,Google,212.0,2.0,lv_200plus,0,0,2,3.8893,0.0000,...,32.0,10412.0,10,1810,3,7,0,0,0,0
6,2211,Apple,227.0,2.0,lv_200plus,0,0,1,0.0000,0.0000,...,5356.0,8769169.0,0,1886,0,0,0,0,0,0
7,2227,Apple,157.0,2.0,lv_140_169,0,0,29,0.0000,0.0000,...,11438.0,1344389.0,0,1157,0,0,0,0,0,0
8,2377,Apple,237.0,3.0,lv_200plus,1,0,1,925.9390,10.4930,...,35.0,436363.0,4,1748,4,0,0,0,0,0
9,2504,Google,119.0,2.0,lv_80_119,0,0,43,0.0000,0.0000,...,826.0,945770.0,0,602,0,0,0,0,0,0


In [10]:
data_for_clustering_5_million = data_for_clustering
data_for_clustering_5_million.to_pickle("Data/data_features_5_million.pkl")

In [23]:
data_for_clustering.to_pickle("data_for_clustering.pkl")

In [13]:
t0= time.time()

data_target_revenue = load_data_from_vertica(query_target_revenue)

t1 = time.time()
print("\nTime elapsed: ", t1-t0)

print()
print(data_target_revenue.shape)
data_target_revenue.head(10)

done vrtc_query

Time elapsed:  3.9297659397125244

(103211, 2)


,dim_user_gid,revenue_converted_net
0,5310,10.4930
1,12020,7.2559
2,12020,2.1712
3,12020,7.2559
4,19909,33.9933
5,20898,19.0053
6,21646,20.4730
7,21646,10.4930
8,25924,89.5930
9,27249,2.0930


In [24]:
data_target_revenue.to_pickle("Data/data_target_revenue.pkl")

In [12]:
t0= time.time()

data_target_level_ups = load_data_from_vertica(query_target_level_ups)

t1 = time.time()
print("\nTime elapsed: ", t1-t0)

print()
print(data_target_level_ups.shape)
data_target_level_ups.head(10)

done vrtc_query

Time elapsed:  4.745867967605591

(178623, 2)


,dim_user_gid,level_ups
0,1004631020,1
1,1004553200,1
2,1001694200,1
3,1004641280,2
4,1004319620,2
5,1004471300,2
6,1004473360,3
7,1003580440,1
8,4249480,1
9,6743080,1


In [23]:
data_target_level_ups.to_pickle("Data/data_target_level_ups.pkl")

In [11]:
t0= time.time()

data_target_mode_number = load_data_from_vertica(query_target_mode_number)

t1 = time.time()
print("\nTime elapsed: ", t1-t0)

print()
print(data_target_mode_number.shape)
data_target_mode_number.head(10)

done vrtc_query

Time elapsed:  66.112717628479

(714479, 2)


,dim_user_gid,typical_modenumber
0,600,5.0
1,800,1.0
2,1223,2.0
3,1646,3.0
4,1735,1.0
5,2191,2.0
6,2447,4.0
7,2472,3.0
8,2576,1.0
9,2717,4.0


In [22]:
data_target_mode_number.to_pickle("Data/data_target_mode_number.pkl")